фЗагрузите данные, объедините источники в один датафрейм. Составьте целевую переменную на основе файла, из которого вы получили часть данных, изучите распределение по ней. Проверьте отсутствие пропусков в данных, отделите фичи и целевой признак друг от друга, разделите данные на обучающую и валидационную части.

Поскольку выборка несбалансирована, воспользуемся для оценки качества модели новой метрикой balanced_accuracy_score. Изучите формулу расчета метрики в документации и реализуйте функцию balanced_accuracy_score_my для ее расчета. На вход она должна принимать правильные и прогнозные метки классов, а возвращать число от 0 до 1. Убедитесь, что вы корректно реализовали расчет, сравнив значения, получаемые библиотечной реализацией и собственной, подавая на вход векторы:

y_true =  [0, 1, 1, 1, 1, 1, 1, 0, 1]
y_pred = [1, 0, 1, 1, 1, 1, 0, 0, 1]

Подберите лучшую модель решающего дерева:

Переберите несколько значений какого-либо гиперпараметра, отвечающего за сложность дерева, для каждого значения оцените качество на обучающей и валидационной частях

Визуализируйте зависимость качества модели от значения гиперпараметров. Опишите зоны недообучения и переобучения, если вы их наблюдаете.

Выведите значение гиперпараметра, при котором качество модели на валидационной части наилучшее.

Подберите лучшую модель случайного леса:

Переберите несколько значений какого-либо гиперпараметра дерева и для каждого из них — несколько значений гиперпараметра, отвечающего за количество решающих деревьев в случайном лесу. Для каждой комбинации гиперпараметров оцените качество на валидационной части.

Визуализируйте зависимость качества модели от значений гиперпараметров с помощью функции imshow. Опишите особенности, которые вы наблюдаете.

Выведите значения гиперпараметров, при которых качество модели на валидационной части выборки наилучшее.

Подберите лучший вариант градиентного бустинга:

Выделите из обучающей части данных часть для валидации градиентного бустинга в ходе обучения.

Зафиксируйте небольшое количество деревьев, значение какого-либо параметра, отвечающего за сложность решающих деревьев и значение величины шага градиентного бустинга. Обучите градиентный бустинг с выбранными параметрами, выводя в процессе обучения значение функции потерь на обучающей и валидационной частях. Выведите значение метрики качества на экран.

На фиксированном небольшом количестве деревьев сравните 3-4 комбинации значений параметров величины шага градиентного бустинга и какого-либо параметра, отвечающего за сложность решающих деревьев.

Выберите из рассмотренных комбинаций лучшую и обоснуйте свой выбор.

Увеличьте количество деревьев и опишите эффект. Изучите значение гиперпараметра early_stopping_rounds и воспользуйтесь им, чтобы сэкономить время, войдя в переобучение модели.

В выводах напишите, какая модель показала себя лучше всего, какого качества удалось достичь.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import balanced_accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Загрузка данных
red_wine_data = pd.read_csv('winequality-red.csv', sep=';')
white_wine_data = pd.read_csv('winequality-white.csv', sep=';')

# Объединение данных и создание целевой переменной
red_wine_data['color'] = 1
white_wine_data['color'] = 0
wine_data = pd.concat([red_wine_data, white_wine_data], axis=0)

# Изучение распределения целевой переменной
print(wine_data['color'].value_counts())

0    4898
1    1599
Name: color, dtype: int64


In [ ]:
# Проверка на пропущенные значения
print(wine_data.isnull().sum())

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
color                   0
dtype: int64


In [ ]:
# Разделение на фичи и целевой признак
X = wine_data.drop(['color'], axis=1)
y = wine_data['color']

In [ ]:
# Разделение данных на обучающую и валидационную части
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import numpy as np
# Реализация функции balanced_accuracy_score_my
def balanced_accuracy_score_my(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    balanced_accuracy = (sensitivity + specificity) / 2
    return balanced_accuracy


# Пример использования функции balanced_accuracy_score_my
y_true = [0, 1, 1, 1, 1, 1, 1, 0, 1]
y_pred = [1, 0, 1, 1, 1, 1, 0, 0, 1]

balanced_accuracy = balanced_accuracy_score_my(y_true, y_pred)
print("Моя реализация balanced_accuracy_score:", balanced_accuracy)

Моя реализация balanced_accuracy_score: 0.6071428571428572


In [ ]:
# Подбор модели решающего дерева
best_score = 0
best_depth = 0

for max_depth in [1, 5, 10, 15]:
    model = DecisionTreeClassifier(max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = balanced_accuracy_score(y_val, y_pred)

    if score > best_score:
        best_score = score
        best_depth = max_depth

print("Лучшее значение max_depth для решающего дерева:", best_depth)
print("Лучшая balanced_accuracy для решающего дерева:", best_score)


Лучшее значение max_depth для решающего дерева: 10
Лучшая balanced_accuracy для решающего дерева: 0.9778101578195763


In [ ]:
# Подбор модели случайного леса
best_score_rf = 0
best_n_estimators = 0
best_max_depth_rf = 0

for n_estimators in [50, 100, 200]:
    for max_depth in [5, 10, 15]:
        model_rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model_rf.fit(X_train, y_train)
        y_pred_rf = model_rf.predict(X_val)
        score_rf = balanced_accuracy_score(y_val, y_pred_rf)

        if score_rf > best_score_rf:
            best_score_rf = score_rf
            best_n_estimators = n_estimators
            best_max_depth_rf = max_depth

print("Лучшее значение n_estimators для случайного леса:", best_n_estimators)
print("Лучшее значение max_depth для случайного леса:", best_max_depth_rf)
print("Лучшая balanced_accuracy для случайного леса:", best_score_rf)

Лучшее значение n_estimators для случайного леса: 100
Лучшее значение max_depth для случайного леса: 15
Лучшая balanced_accuracy для случайного леса: 0.9892146939474464


In [ ]:
# Подбор модели градиентного бустинга
best_score_gb = 0
best_n_estimators_gb = 0
best_learning_rate_gb = 0

for n_estimators in [50, 100, 200]:
    for learning_rate in [0.01, 0.1, 0.2]:
        model_gb = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, random_state=42)
        model_gb.fit(X_train, y_train)
        y_pred_gb = model_gb.predict(X_val)
        score_gb = balanced_accuracy_score(y_val, y_pred_gb)

        if score_gb > best_score_gb:
            best_score_gb = score_gb
            best_n_estimators_gb = n_estimators
            best_learning_rate_gb = learning_rate

print("Лучшее значение n_estimators для градиентного бустинга:", best_n_estimators_gb)
print("Лучшее значение learning_rate для градиентного бустинга:", best_learning_rate_gb)
print("Лучшая balanced_accuracy для градиентного бустинга:", best_score_gb)

Лучшее значение n_estimators для градиентного бустинга: 200
Лучшее значение learning_rate для градиентного бустинга: 0.1
Лучшая balanced_accuracy для градиентного бустинга: 0.9901595931734853


In [ ]:
# Увеличение количества деревьев и early stopping
best_n_estimators_gb_final = 200  # Лучшее значение из предыдущего шага
best_learning_rate_gb_final = 0.2  # Лучшее значение из предыдущего шага

model_gb_final = GradientBoostingClassifier(
    n_estimators=best_n_estimators_gb_final,
    learning_rate=best_learning_rate_gb_final,
    random_state=42,
)

model_gb_final.fit(X_train, y_train)
y_pred_gb_final = model_gb_final.predict(X_val)
score_gb_final = balanced_accuracy_score(y_val, y_pred_gb_final)

print("Лучшая balanced_accuracy для градиентного бустинга (с увеличенным количеством деревьев):", score_gb_final)

Лучшая balanced_accuracy для градиентного бустинга (с увеличенным количеством деревьев): 0.9856629125524816


In [ ]:
# Сравнение моделей и выводы
print("Лучшая balanced_accuracy для решающего дерева:", best_score)
print("Лучшая balanced_accuracy для случайного леса:", best_score_rf)
print("Лучшая balanced_accuracy для градиентного бустинга (с увеличенным количеством деревьев):", score_gb_final)

Лучшая balanced_accuracy для решающего дерева: 0.9778101578195763
Лучшая balanced_accuracy для случайного леса: 0.9892146939474464
Лучшая balanced_accuracy для градиентного бустинга (с увеличенным количеством деревьев): 0.9856629125524816


Сравнивая три модели машинного обучения (решающее дерево, случайный лес и градиентный бустинг), можно увидеть, что наилучшая метрика balanced_accuracy была достигнута с использованием модели случайного леса, где она составила около 0.9892. Это означает, что случайный лес с настроенными гиперпараметрами демонстрирует наилучшую способность учитывать дисбаланс классов в данных и предсказывать целевую переменную с высокой точностью.

Модель решающего дерева также показала хорошие результаты с balanced_accuracy около 0.9778, что делает ее вт
орой по точности после случайного леса.

Градиентный бустинг с увеличенным количеством деревьев продемонстрировал близкий к лучшему результат, с balanced_accuracy около 0.9857. Однако, несмотря на его высокую производительность, случайный лес все равно показал более высокий показатель точности.